## Multiple Doc query

In [4]:
## Loading all the Documents from the directory

from langchain.document_loaders import DirectoryLoader
dir_path =  r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\3_Text_query_bot\_docs\to_upload_docs\pdf_documents"
loader = DirectoryLoader(path= dir_path , show_progress=True)
docs = loader.load()
len(docs)

  0%|          | 0/7 [00:00<?, ?it/s]c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 7/7 [00:34<00:00,  4.98s/it]


7

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter =RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200 )
texts  = text_splitter.split_documents(docs)
len(texts)

2355

In [6]:
## Persisting Directory
# to store the embeddings on the disk
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embed_llm_path = r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models\embedding_models\snowflake-arctic-embed-m"
embed_llm =  HuggingFaceEmbeddings(
            model_name = embed_llm_path,
            show_progress = True,
            model_kwargs = {"trust_remote_code": True})

persist_directory = r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\3_Text_query_bot\_docs\to_upload_docs\doc_db"

# vectorDB = Chroma.from_documents(documents= texts, embedding= embed_llm, persist_directory= persist_directory)


You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [7]:
vectorDB =  None
vectorDB = Chroma(persist_directory= persist_directory, embedding_function= embed_llm)
retriever = vectorDB.as_retriever(search_kwargs= {'k':4})
ret_docs = retriever.get_relevant_documents("What are types of leaves discussed in Leave Policy document?")
ret_docs

c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Batches: 100%|██████████| 1/1 [00:00<00:00,  1.62it/s]


[Document(page_content='2.8 Special Leave: In addition, an employee may be entitled to leave under specific conditions such as Parental Leave (Maternity/ Paternity), Special Leave on joining or transfer, Sabbatical Leave, etc.\n\n2.9 Public Holiday: Every year, ten public holidays are declared by businesses for the relevant locations to incorporate local customs. Businesses/locations can choose to give two optional holidays out of ten holidays to make the policy more inclusive. Employees will have the option to choose two holidays from the list of Optional holidays (published by HR SSC at the start of the calendar year). This number is independent of the date of joining. Employees will have to choose their preferred holidays from the proposed optional holiday. If Optional holidays are not chosen by any employee in a given year, those holidays will lapse for the year.\n\nPage 1 of 4\n\nLeave Policy\n\nVersion Issue Date\n\nHRLP24 1-Apr-2024\n\nEffective Date\n\n1-Jan-2024', metadata={'s

In [8]:
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_community.llms.ctransformers import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


## setting the temp = 0
llm_dir = r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models"
llm_file = "Mistral-7B-Instruct-v0.3.Q4_K_M.gguf"

llm = CTransformers( model= llm_dir, model_file = llm_file, callbacks=[StreamingStdOutCallbackHandler()], config = {"context_length": 16000, "max_new_tokens": 3000, "temperature":0})
 
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm= llm, 
    chain_type= "stuff",
    retriever = retriever,
    return_source_documents =  True
)

 
 

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
                    
def get_response(llm, retriever, query):
    qa_system_prompt = """
    You must answer the user's questions. \
    Use the following pieces of retrieved context to answer the question :{context} \
    If you don't know the answer, just say that you don't know. \
    Do not make up fake information. \
    Keep the answer concise.\

    Answer: """

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", qa_system_prompt),
            ("human", "{input}"),
        ]
    )
            
    rag_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | qa_prompt
    | llm)
        
    return rag_chain.invoke(query)    

In [14]:
# response = qa_chain.invoke("Leave policy 2024")
query = "When does the leave year start as discussed in Leave Policy?"
response = get_response(llm, retriever, query)
response

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 16.61it/s]



Assistant: The leave year starts on the 1st of January and ends on the 31st of December, according to the Leave Policy.

'\nAssistant: The leave year starts on the 1st of January and ends on the 31st of December, according to the Leave Policy.'

 In Lewis Carroll's Alice in Wonderland, young Alice finds a magical world hidden behind her own. While chasing the White Rabbit who appears to be late for an important meeting, she falls down a rabbit hole and enters Wonderland, where she encounters various fantastical creatures like the Cheshire Cat, the Mad Hatter, and the Queen of Hearts. Throughout the story, Alice grows confused as Wonderland's strange logic defies her expectations and experiences the chaotic Wonderland tea party. Alice often tries to make sense of the nonsensical world around her, leading to humorous exchanges with the characters she meets. The narrative continues with more peculiarities in Alice Through the Looking-Glass, which serves as a sequel. In total, both books present an imaginative and whimsical journey that challenges conventional wisdom and reality.
